In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans

In [2]:
sale1= pd.read_csv('SalesData_AB_fixed.csv', low_memory=False)
sale2= pd.read_csv('SalesData_-BC&MB_fixed.csv', low_memory= False)
sale3= pd.read_csv('SalesData_-SK_fixed.csv', low_memory=False)
farmo = pd.read_csv('Farmo_pivot.csv')

In [3]:
print(sale1.shape, sale2.shape, sale3.shape)

(104433, 19) (67852, 19) (104320, 19)


In [4]:
df = pd.concat([sale1,sale2,sale3], axis=0)

In [5]:
df.shape

(276605, 19)

In [6]:
df.columns

Index(['Authorization', 'Grower_AIN', 'city_name', 'province_name',
       'Dealer_AIN', 'Dealer_City_Name', 'Dealer_Prov', 'Crop Year', 'Season',
       'Business', 'Product Category', 'Product Brand', 'Rolled product name',
       'SumOfSumOfConverted Quantity', 'SumOfSumOfConverted Acres',
       'SumOfSumOfConverted Total_MSRP', 'PHI_area_cd', 'PHI_district_cd',
       'PHI_territory_cd'],
      dtype='object')

In [7]:
df = df.rename(columns={'SumOfSumOfConverted Quantity':'Quantity', 'SumOfSumOfConverted Acres':'Acres', 
                       'SumOfSumOfConverted Total_MSRP':'MSRP', 'Crop Year':'Year', 'Product Category':'Category'
              , 'Product Brand':'Brand', 'Rolled product name':'product_name'})

In [8]:
df.head(10)

,Authorization,Grower_AIN,city_name,province_name,Dealer_AIN,Dealer_City_Name,Dealer_Prov,Year,Season,Business,Category,Brand,product_name,Quantity,Acres,MSRP,PHI_area_cd,PHI_district_cd,PHI_territory_cd
0,Authorized,CA14EU-0025528,Camrose,AB,CA18R-1167094,Camrose,AB,2018,Spring,Crop Protection,Cereal BL,Cirpreme,Cirpreme XC 0.8KG+ 4.1L Case,7.000,560.00,5852.00,NaN,NaN,NaN
1,Authorized,CA14EU-0025528,Camrose,AB,CA18R-1167094,Camrose,AB,2018,Spring,Crop Protection,Cereal BL,Pixxaro,Pixxaro 1 X (9.45 L+ 4.9L) case,33.000,1320.00,14638.80,NaN,NaN,NaN
2,Authorized,CA14EU-0025528,Camrose,AB,CA18R-1167094,Camrose,AB,2018,Spring,Crop Protection,Pre-Seed,PrePass FLEX,PrePass Flex 8 x 0.648kg case,0.625,400.00,1800.00,NaN,NaN,NaN
3,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2016,Spring,Crop Protection,Cereal BL,Prestige XC,Prestige XC 19.3L Case,11.000,293.37,3797.72,NaN,NaN,NaN
4,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2016,Spring,Crop Protection,Cereal BL,Prestige XC,Prestige XC 463.2L Pallet,2.750,1760.22,22786.30,NaN,NaN,NaN
5,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2016,Spring,Crop Protection,Cereal Grass,Simplicity,Simplicity 19L Case,2.000,160.00,3320.00,NaN,NaN,NaN
6,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2016,Spring,Crop Protection,Pre-Seed,PrePass XC,PrePass XC 16.6L Case,9.000,360.00,2574.00,NaN,NaN,NaN
7,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2017,Spring,Crop Protection,Cereal BL,Prestige XC,Prestige XC 19.3L Case,32.000,853.44,11261.19,NaN,NaN,NaN
8,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2017,Spring,Crop Protection,Cereal BL,Prestige XL,Prestige XL 113.6L Drum,5.000,800.00,10608.00,NaN,NaN,NaN
9,Authorized,CA14EU-0025529,Ohaton,AB,CA18R-1167094,Camrose,AB,2017,Spring,Crop Protection,Cereal BL,Spectrum,Spectrum 12.8L Case,31.000,620.00,9913.80,NaN,NaN,NaN


In [9]:
df = df.drop(columns=['PHI_area_cd', 'PHI_district_cd','PHI_territory_cd'])
df.isnull().sum()

Authorization          0
Grower_AIN             0
city_name              0
province_name          0
Dealer_AIN             0
Dealer_City_Name       0
Dealer_Prov            0
Year                   0
Season              2588
Business               0
Category               0
Brand                  0
product_name           0
Quantity               0
Acres                  0
MSRP                8851
dtype: int64

In [10]:
print(df.Authorization.unique(), "unique customers:", df.Grower_AIN.nunique(), ",","province"
      , df.province_name.nunique(),",","dealer:", df.Dealer_AIN.nunique(), "," , df.Year.unique(), df.Season.unique()
      , df.Business.unique(), "Categories:", df.Category.nunique(), ",", "brand:", df.Brand.nunique(), ","
      , "product_name:" ,df.product_name.nunique())

['Authorized' 'Unauthorized'] unique customers: 59388 , province 14 , dealer: 1091 , [2018 2016 2017] ['Spring' nan 'Fall'] ['Crop Protection' 'Brevant' 'Pioneer' 'Specialties'] Categories: 22 , brand: 92 , product_name: 441


In [11]:
farmo1 = farmo[['AIN', 'Actual Total Seeded Area']]
profile_df = pd.merge(df, farmo1, left_on='Grower_AIN' ,right_on='AIN' , how='inner')
print(profile_df.shape)

(185981, 18)


In [12]:
print(profile_df.Authorization.unique(), "unique customers:", profile_df.Grower_AIN.nunique(), ",","province"
      , profile_df.province_name.nunique(),",","dealer:", profile_df.Dealer_AIN.nunique(), "," 
      , profile_df.Year.unique(), profile_df.Season.unique(), profile_df.Business.unique(), "Categories:"
      , profile_df.Category.nunique(), ",", "brand:", profile_df.Brand.nunique(), ","
      , "product_name:" ,profile_df.product_name.nunique())

['Authorized'] unique customers: 24108 , province 4 , dealer: 1077 , [2018 2016 2017] ['Spring' 'Fall' nan] ['Crop Protection' 'Pioneer' 'Specialties' 'Brevant'] Categories: 22 , brand: 91 , product_name: 430


In [13]:
profile_df.isnull().sum()

Authorization                  0
Grower_AIN                     0
city_name                      0
province_name                  0
Dealer_AIN                     0
Dealer_City_Name               0
Dealer_Prov                    0
Year                           0
Season                      2480
Business                       0
Category                       0
Brand                          0
product_name                   0
Quantity                       0
Acres                          0
MSRP                        6918
AIN                            0
Actual Total Seeded Area       0
dtype: int64

In [14]:
profile_df[profile_df.Season.isnull()]

,Authorization,Grower_AIN,city_name,province_name,Dealer_AIN,Dealer_City_Name,Dealer_Prov,Year,Season,Business,Category,Brand,product_name,Quantity,Acres,MSRP,AIN,Actual Total Seeded Area
168,Authorized,CA14EU-0006044,Hardisty,AB,CA14R-0005742,Camrose,AB,2016,NaN,Brevant,Corn,Corn,Corn_BAXXOSRR F1 Bag Poncho 250/Maxim/Apron/Dy...,50.0,125.0,12050.0,CA14EU-0006044,750.0
216,Authorized,CA14EU-0006094,Stettler,AB,CA14R-0005742,Camrose,AB,2016,NaN,Brevant,Corn,Corn,Corn_BAXXOSRR F1 Bag Poncho 250/Maxim/Apron/Dy...,2.0,5.0,482.0,CA14EU-0006094,1000.0
451,Authorized,CA14EU-0006471,Paradise Valley,AB,CA17R-0898885,Wainwright,AB,2016,NaN,Brevant,Corn,Corn,Corn_BAXXOSRR R2 Bag Poncho 250/Maxim/Apron/Dy...,14.0,35.0,3374.0,CA14EU-0006471,453.0
452,Authorized,CA14EU-0006471,Paradise Valley,AB,CA17R-0898885,Wainwright,AB,2016,NaN,Brevant,Corn,Corn,Corn_DS80A27 F1 Bag CruiserMaxx Corn 250 - ON ...,1.0,2.5,279.0,CA14EU-0006471,453.0
966,Authorized,CA14EU-0009898,Lloydminster,SK,CA17R-0898885,Wainwright,AB,2016,NaN,Brevant,Corn,Corn,Corn_4093 F1 Bag CruiserMaxx Corn 250 - ON Cla...,1.0,2.5,269.0,CA14EU-0009898,1067.0
967,Authorized,CA14EU-0009898,Lloydminster,SK,CA17R-0898885,Wainwright,AB,2016,NaN,Brevant,Corn,Corn,Corn_4093 F2 Bag CruiserMaxx Corn 250 - ON Cla...,1.0,2.5,269.0,CA14EU-0009898,1067.0
968,Authorized,CA14EU-0009898,Lloydminster,SK,CA17R-0898885,Wainwright,AB,2016,NaN,Brevant,Corn,Corn,Corn_BAXXOSRR F1 Bag Poncho 250/Maxim/Apron/Dy...,24.0,60.0,5784.0,CA14EU-0009898,1067.0
1555,Authorized,CA14EU-0011886,Coaldale,AB,CA14R-0005767,Coaldale,AB,2016,NaN,Brevant,Corn,Corn,Corn_4085 F2 Bag CruiserMaxx Corn 250 - ON Cla...,3.0,7.5,822.0,CA14EU-0011886,8300.0
1556,Authorized,CA14EU-0011886,Coaldale,AB,CA14R-0005767,Coaldale,AB,2016,NaN,Brevant,Corn,Corn,Corn_TMF8106RA R1 Bag CruiserMaxx Corn 250 - O...,9.0,22.5,2466.0,CA14EU-0011886,8300.0
1557,Authorized,CA14EU-0011886,Coaldale,AB,CA14R-0005767,Coaldale,AB,2017,NaN,Brevant,Corn,Corn,Corn_TMF8106RA F1 Bag CruiserMaxx Corn 250 - O...,36.0,90.0,9504.0,CA14EU-0011886,8300.0


In [15]:
profile_df[profile_df.Brand == 'Corn'].Season.unique()

array(['Spring', nan], dtype=object)

In [16]:
profile_df.Season[profile_df.Season.isnull()] = "Spring"
print("missing values:", profile_df.Season[profile_df.Season.isnull()].count())

missing values: 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
profile_df.dropna(subset=['MSRP'], how='all', inplace = True)

In [18]:
profile_df.isnull().sum() 

Authorization               0
Grower_AIN                  0
city_name                   0
province_name               0
Dealer_AIN                  0
Dealer_City_Name            0
Dealer_Prov                 0
Year                        0
Season                      0
Business                    0
Category                    0
Brand                       0
product_name                0
Quantity                    0
Acres                       0
MSRP                        0
AIN                         0
Actual Total Seeded Area    0
dtype: int64

In [19]:
profile_df.shape

(179063, 18)